In [3]:
import re
import pandas as pd
from matplotlib import pyplot as plt

In [4]:
# 不用儲存article_id, 轉成list
def training_txt_to_list(path):
    with open(path, 'r') as f:
        txt = str(f.read())
    txt_list = txt.split('\n')
    text_label_list = list()
    tmp = list()
    for line in txt_list:
        if line == '--------------------':
            text_label_list.append(tmp)
            tmp = list()
            continue
        if line == '':
            continue
        tmp.append(line)
    x = list()
    y = list()
    for text_label in text_label_list:
        text = text_label[0]
        label = text_label[2:]
        label_list = ['O' for i in range(len(text))]
        for i in label:
            entity = i.split('\t')
            if int(entity[1]) > int(entity[2]):
                continue
            b = int(entity[1])
            label_list[b] = 'B-{}'.format(entity[-1])
            for j in range(int(entity[1])+1, int(entity[2])):
                label_list[j] = 'I-{}'.format(entity[-1])
        for i, j in enumerate(text):
            if j == '，' or j == '。' or j == '？':
                label_list[i] = j
                
        text_list = re.split('\uff0c|\u3002|\uff1f', text)
        for sentence in text_list:
            x.append([i for i in sentence])
        x = x[:-1]
            
        sentence = list()
        for i in label_list:
            if i == '，' or i == '。' or i == '？':
                y.append(sentence)
                sentence = list()
            else:
                sentence.append(i)
                
    return [x, y]

In [7]:
x1, y1 = training_txt_to_list('/home/Danny/AI-CUP-2020/datasets/stage1/SampleData_deid.txt')
x2, y2 = training_txt_to_list('/home/Danny/AI-CUP-2020/datasets/stage2/train_1_update.txt')
x4, y4 = training_txt_to_list('/home/Danny/AI-CUP-2020/datasets/stage4/train_2.txt')

In [8]:
x = x1 + x2 + x4
y = y1 + y2 + y4
print(len(x))
print(len(y))

76162
76162


In [16]:
df = pd.DataFrame({'x': x, 'y': y})

In [17]:
df['x'] = df['x'].apply(lambda row : str(row))
df['y'] = df['y'].apply(lambda row : str(row))
df = df.drop_duplicates()
df['x'] = df['x'].apply(lambda row : eval(row))
df['y'] = df['y'].apply(lambda row : eval(row))
df

,x,y
0,"[醫, 師, ：, 你, 有, 做, 超, 音, 波, 嘛]","[O, O, O, O, O, O, O, O, O, O]"
1,"[那, 我, 們, 來, 看, 報, 告]","[O, O, O, O, O, O, O]"
2,"[有, 些, 部, 分, 有, 紅, 字, 耶]","[O, O, O, O, O, O, O, O]"
3,"[民, 眾, ：, 紅, 字, 是, 甚, 麼, 意, 思]","[O, O, O, O, O, O, O, O, O, O]"
4,"[醫, 師, ：, 就, 是, 肝, 功, 能, 有, 比, 較, 高]","[O, O, O, O, O, O, O, O, O, O, O, O]"
...,...,...
75713,"[醫, 師, ：, 7, 4, 0, 3]","[O, O, O, B-ID, I-ID, I-ID, I-ID]"
75714,"[醫, 師, ：, 6, 6, 6, 0, 8, 9, 7, 4, 0, 3]","[O, O, O, B-ID, I-ID, I-ID, I-ID, I-ID, I-ID, ..."
75716,"[護, 理, 師, ：, 6, 6, 6, 0, 9, 8]","[O, O, O, O, B-ID, I-ID, I-ID, I-ID, I-ID, I-ID]"
75717,"[醫, 師, ：, 0, 9, 8, 7, 4, 0, 3]","[O, O, O, B-ID, I-ID, I-ID, I-ID, I-ID, I-ID, ..."


In [18]:
df = df.reset_index(drop=True)
df

,x,y
0,"[醫, 師, ：, 你, 有, 做, 超, 音, 波, 嘛]","[O, O, O, O, O, O, O, O, O, O]"
1,"[那, 我, 們, 來, 看, 報, 告]","[O, O, O, O, O, O, O]"
2,"[有, 些, 部, 分, 有, 紅, 字, 耶]","[O, O, O, O, O, O, O, O]"
3,"[民, 眾, ：, 紅, 字, 是, 甚, 麼, 意, 思]","[O, O, O, O, O, O, O, O, O, O]"
4,"[醫, 師, ：, 就, 是, 肝, 功, 能, 有, 比, 較, 高]","[O, O, O, O, O, O, O, O, O, O, O, O]"
...,...,...
31516,"[醫, 師, ：, 7, 4, 0, 3]","[O, O, O, B-ID, I-ID, I-ID, I-ID]"
31517,"[醫, 師, ：, 6, 6, 6, 0, 8, 9, 7, 4, 0, 3]","[O, O, O, B-ID, I-ID, I-ID, I-ID, I-ID, I-ID, ..."
31518,"[護, 理, 師, ：, 6, 6, 6, 0, 9, 8]","[O, O, O, O, B-ID, I-ID, I-ID, I-ID, I-ID, I-ID]"
31519,"[醫, 師, ：, 0, 9, 8, 7, 4, 0, 3]","[O, O, O, B-ID, I-ID, I-ID, I-ID, I-ID, I-ID, ..."


In [31]:
def find_contain_family_entity_rows(labels):
    dic_temp = {}
    dummy = 0
    for label in labels:
        dic_temp[label] = dummy
    #if len(dic_temp) > 1:
    #    return True
    if 'B-family' in dic_temp:
        return True
    else:
        return False
df['flag'] = df['y'].apply(lambda l:find_contain_family_entity_rows(l))

In [32]:
df

,x,y,flag
0,"[醫, 師, ：, 你, 有, 做, 超, 音, 波, 嘛]","[O, O, O, O, O, O, O, O, O, O]",False
1,"[那, 我, 們, 來, 看, 報, 告]","[O, O, O, O, O, O, O]",False
2,"[有, 些, 部, 分, 有, 紅, 字, 耶]","[O, O, O, O, O, O, O, O]",False
3,"[民, 眾, ：, 紅, 字, 是, 甚, 麼, 意, 思]","[O, O, O, O, O, O, O, O, O, O]",False
4,"[醫, 師, ：, 就, 是, 肝, 功, 能, 有, 比, 較, 高]","[O, O, O, O, O, O, O, O, O, O, O, O]",False
...,...,...,...
31516,"[醫, 師, ：, 7, 4, 0, 3]","[O, O, O, B-ID, I-ID, I-ID, I-ID]",False
31517,"[醫, 師, ：, 6, 6, 6, 0, 8, 9, 7, 4, 0, 3]","[O, O, O, B-ID, I-ID, I-ID, I-ID, I-ID, I-ID, ...",False
31518,"[護, 理, 師, ：, 6, 6, 6, 0, 9, 8]","[O, O, O, O, B-ID, I-ID, I-ID, I-ID, I-ID, I-ID]",False
31519,"[醫, 師, ：, 0, 9, 8, 7, 4, 0, 3]","[O, O, O, B-ID, I-ID, I-ID, I-ID, I-ID, I-ID, ...",False


In [33]:
df_entity = df[df['flag'] == True]

In [35]:
df_entity.reset_index(drop=True)

,x,y,flag
0,"[我, 哥, 哥, 帶, 我, 來, 的, 他, 就, 回, 去, 了]","[O, B-family, I-family, O, O, O, O, O, O, O, O...",True
1,"[民, 眾, ：, 我, 3, 個, 叔, 叔, 都, 五, 十, 幾, 歲, 就, …, ...","[O, O, O, O, O, O, B-family, I-family, O, B-ti...",True
2,"[家, 屬, ：, 因, 為, 我, 爸, 爸, 住, 院, 的, 時, 候, 他, 當, ...","[O, O, O, O, O, B-family, I-family, I-family, ...",True
3,"[家, 屬, ：, 啊, 常, 常, 照, 顧, 我, 爸]","[O, O, O, O, O, O, O, O, B-family, I-family]",True
4,"[我, 大, 伯, 有, 一, 個, 問, 題, 是]","[O, B-family, I-family, O, O, O, O, O, O]",True
5,"[爸, 爸, 他, 請, 我, 帶, 他, 去, 屏, 東, 做, 這, 個]","[B-family, I-family, O, O, O, O, O, O, B-locat...",True
6,"[因, 為, 是, 姐, 姐, 在, 做, 月, 子, 我, 也, 跟, 著, 在, 坐, ...","[O, O, O, B-family, I-family, O, O, O, O, O, O...",True
7,"[民, 眾, ：, 因, 為, 我, …, …, 我, 大, 姊]","[O, O, O, O, O, O, O, O, O, B-family, I-family]",True
8,"[我, 大, 姊, 就, 是, 肝]","[O, B-family, I-family, O, O, O]",True
9,"[二, 女, 兒, 跟, 小, 女, 兒, 三, 個, 她, 們, 都, 有, C, 肝]","[B-family, I-family, I-family, O, B-family, I-...",True


In [39]:
family_df = pd.read_csv('../dictionary/online/family.csv')

In [72]:
aug_df = pd.DataFrame([],columns=['x','y'])
aug_df

,x,y


In [73]:
for index, row in df_entity.iterrows():
    entity_start_pos = 0
    entity_end_pos = -1
    start_flag = False
    for i in range(len(row['y'])):
        if row['y'][i] == 'B-family':
            entity_start_pos = i
            start_flag = True
        elif row['y'][i] == 'O' and start_flag == True:
            entity_end_pos = i
            break
        else:
            continue
    for entity in family_df['entity_text']:
        sentence = []
        labels = []
        count = 0
        if '我' not in entity and '你' not in entity and '他' not in entity:
            for i in range(entity_start_pos):
                sentence.append(row['x'][i])
                labels.append('O')
            sentence.append(entity[0])
            labels.append('B-family')
            for i in range(1,len(entity)):
                sentence.append(entity[i])
                labels.append('I-family')
            if entity_end_pos != -1:
                for i in range(entity_end_pos,len(row['x'])):
                    sentence.append(row['x'][i])
                    labels.append('O')
            #print(sentence)
            #print(labels)
            if len(sentence) != len(labels):
                print('Wrong')
            aug_df = aug_df.append({'x':sentence,'y':labels},ignore_index=True)
    

In [74]:
aug_df

,x,y
0,"[我, 堂, 姐, 帶, 我, 來, 的, 他, 就, 回, 去, 了]","[O, B-family, I-family, O, O, O, O, O, O, O, O..."
1,"[我, 堂, 姊, 帶, 我, 來, 的, 他, 就, 回, 去, 了]","[O, B-family, I-family, O, O, O, O, O, O, O, O..."
2,"[我, 堂, 妹, 帶, 我, 來, 的, 他, 就, 回, 去, 了]","[O, B-family, I-family, O, O, O, O, O, O, O, O..."
3,"[我, 堂, 哥, 帶, 我, 來, 的, 他, 就, 回, 去, 了]","[O, B-family, I-family, O, O, O, O, O, O, O, O..."
4,"[我, 堂, 弟, 帶, 我, 來, 的, 他, 就, 回, 去, 了]","[O, B-family, I-family, O, O, O, O, O, O, O, O..."
...,...,...
4601,"[他, 娘, 那, 邊, 可, 能, 會, 有, 壓, 力, 啦]","[O, B-family, O, O, O, O, O, O, O, O, O]"
4602,"[他, 祖, 父, 那, 邊, 可, 能, 會, 有, 壓, 力, 啦]","[O, B-family, I-family, O, O, O, O, O, O, O, O..."
4603,"[他, 外, 祖, 父, 那, 邊, 可, 能, 會, 有, 壓, 力, 啦]","[O, B-family, I-family, I-family, O, O, O, O, ..."
4604,"[他, 祖, 母, 那, 邊, 可, 能, 會, 有, 壓, 力, 啦]","[O, B-family, I-family, O, O, O, O, O, O, O, O..."


In [75]:
aug_df = aug_df.sample(500)
aug_df = aug_df.reset_index(drop=True)
aug_df

,x,y
0,"[他, 娘, 那, 邊, 可, 能, 會, 有, 壓, 力, 啦]","[O, B-family, O, O, O, O, O, O, O, O, O]"
1,"[三, 叔, 跟, 小, 女, 兒, 三, 個, 她, 們, 都, 有, C, 肝]","[B-family, I-family, O, O, O, O, O, O, O, O, O..."
2,"[家, 屬, ：, 啊, 我, 哥, 哥, 說, 嚇, 到, 阿]","[O, O, O, O, O, B-family, I-family, O, O, O, O]"
3,"[家, 屬, ：, 啊, 常, 常, 照, 顧, 媽, 媽]","[O, O, O, O, O, O, O, O, B-family, I-family]"
4,"[他, 大, 舅, 舅, 開, 刀]","[O, B-family, I-family, I-family, O, O]"
...,...,...
495,"[阿, 不, 然, 拜, 託, 你, 伯, 父, 幫, 你, 拿]","[O, O, O, O, O, O, B-family, I-family, O, O, O]"
496,"[我, 跟, 我, 祖, 母, 說, 我, 晚, 上, 都, 睡, 不, 著]","[O, O, O, B-family, I-family, O, O, O, O, O, O..."
497,"[因, 為, 是, 表, 弟, 在, 做, 月, 子, 我, 也, 跟, 著, 在, 坐, ...","[O, O, O, B-family, I-family, O, O, O, O, O, O..."
498,"[民, 眾, ：, 這, 個, 是, 後, 面, 爸, 爸, 帶, 我, 去, ⋯, ⋯, ...","[O, O, O, O, O, O, O, O, B-family, I-family, O..."


In [76]:
aug_df.to_csv('../datasets/family_augmentation.csv',index=False)